In [1]:
import numpy as np
import matplotlib.pyplot as plt


from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, log_loss, accuracy_score

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense,Flatten,GlobalAveragePooling2D,Conv2D,MaxPooling2D,Activation,Dropout


import warnings
warnings.filterwarnings('ignore')

from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod, CarliniLInfMethod
from art.attacks.evasion import FeatureAdversariesTensorFlowV2
from art.estimators.classification import TensorFlowV2Classifier

In [9]:
lfw_dataset=fetch_lfw_people(min_faces_per_person=100)

print(lfw_dataset.images.shape)
print(lfw_dataset.data.shape)
print(lfw_dataset.target_names.shape)
print(lfw_dataset.target.shape)

lfw_dataset.target

Name = lfw_dataset.target_names

print(Name)
print(len(Name))

N=[]
for i in range(len(Name)):
    N+=[i]
    
mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

def mapper(value):
    return reverse_mapping[value]

X0=lfw_dataset.images
y=lfw_dataset.target

print(X0.shape)
X=X0.reshape(-1,62,47,1)
print(X.shape)

dataset=[]
testset=[]
t=0
for Xi,yi in zip(X,y):
    img=Xi/255.0
    if t<=200:
        dataset.append([img,yi])
    else:   
        testset.append([img,yi])
    t+=1

data,labels0=zip(*dataset)
test,tlabels0=zip(*testset)

labels1=to_categorical(labels0)
data=np.array(data)
labels=np.array(labels1)

tlabels1=to_categorical(tlabels0)
test=np.array(test)
tlabels=np.array(tlabels1)

# trainx,testx,trainy,testy=train_test_split(data,labels,test_size=0.2,random_state=44)
trainx,testx,trainy,testy=train_test_split(data,labels,test_size=0.2,random_state=12)

trainx = trainx.astype(np.float32)
testx = testx.astype(np.float32)
trainy = trainy.astype(np.float32)
testy = testy.astype(np.float32)

print(trainx.shape)
print(testx.shape)
print(trainy.shape)
print(testy.shape)

(1140, 62, 47)
(1140, 2914)
(5,)
(1140,)
['Colin Powell' 'Donald Rumsfeld' 'George W Bush' 'Gerhard Schroeder'
 'Tony Blair']
5
(1140, 62, 47)
(1140, 62, 47, 1)
(160, 62, 47, 1)
(41, 62, 47, 1)
(160, 5)
(41, 5)


In [2]:
# datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.2,
#                     width_shift_range=0.2,height_shift_range=0.2,shear_range=0.1,fill_mode="nearest")

model = Sequential()

model.add(Conv2D(32,(3,3), input_shape=(62,47,1), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(32,(3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=5, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 60, 45, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 30, 22, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 20, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 10, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 4480)              0         
                                                                 
 dense (Dense)               (None, 512)               2

In [3]:

# model.save("VGG_model_85.pth")
# !zip -r VGG_model_85.zip VGG_model_85.pth
# model = keras.models.load_model("VGG_model_85.pth")

## Training model

# his=model.fit(datagen.flow(trainx,trainy,batch_size=32),validation_data=(testx,testy),epochs=1000)
his=model.fit(trainx,trainy, validation_data=(testx,testy),epochs=100, batch_size=32)

NameError: name 'trainx' is not defined

In [26]:
# Step 5: Evaluate the ART classifier on benign test examples

predictions = model.predict(testx)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(testy, axis=1)) / len(testy)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Accuracy on benign test examples: 73.17073170731707%


2023-05-10 12:26:11.709716: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [35]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss_object = tf.keras.losses.CategoricalCrossentropy()

def train_step(model, images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

classifier = TensorFlowV2Classifier(
    model=model,
    loss_object=loss_object,
    train_step=train_step,
    nb_classes=5,
    input_shape=(62, 47, 1),
    clip_values=(0, 1),
)

In [36]:
testx.shape, trainx.shape, type(testx), type(trainx)

((41, 62, 47, 1), (160, 62, 47, 1), numpy.ndarray, numpy.ndarray)

In [37]:
predictions = classifier.predict(testx)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(testy, axis=1)) / len(testy)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

AttributeError: 'Tensor' object has no attribute 'numpy'

In [ ]:
# Step 5: prepare a batch of source and guide images
valid = np.argmax(testy, axis=1)[:20] != np.argmax(testy, axis=1)[20:40]
source = testx[:20][valid][:32]
guide = testx[20:40][valid][:32]

In [ ]:
valid.shape, source.shape, guide.shape

In [ ]:
from art.attacks.evasion import FeatureAdversariesTensorFlowV2
from art.estimators.classification import TensorFlowV2Classifier

# Step 6: Generate adversarial test examples
attack = FeatureAdversariesTensorFlowV2(
    classifier,
    layer=-2,
    delta=45/255,
    optimizer=None,
    step_size=1/255,
    max_iter=100,
)
x_test_adv = attack.generate(source, guide)

# Step 7: Evaluate the ART classifier on adversarial test examples


In [ ]:
predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(testy[:20][valid][:32], axis=1)) / len(testy[:20][valid][:32])

dim = tuple(range(1, len(source.shape)))
pert = np.mean(np.amax(np.abs(source - x_test_adv), axis=dim))
print("Accuracy on adversarial test batch: {}%".format(accuracy * 100))
print("Average perturbation: {}%".format(pert))

In [ ]:
# eps = [1e-5, 1/255, 2/255, 3/255]
eps = np.linspace(1e-3, 20/255, num=100)
accuracyList = []
for _eps in eps:
    attack = FeatureAdversariesTensorFlowV2(
        classifier,
        layer=-2,
        delta=_eps,
        optimizer=None,
        step_size=1/255,
        max_iter=100,
    )
    x_test_adv = attack.generate(source, guide)
    predictions = classifier.predict(x_test_adv)
    accuracyList.append(np.sum(np.argmax(predictions, axis=1) == np.argmax(testy[:20][valid][:32], axis=1)) / len(testy[:20][valid][:32]))
    print("finish with eps:", _eps)

In [ ]:
plt.plot(eps, accuracyList)

In [ ]:
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import TensorFlowV2Classifier

In [ ]:
estimator = FeatureAdversariesTensorFlowV2(
        classifier,
        layer=-2,
        delta=45/255,
        optimizer=tf.keras.optimizers.Adam,
        optimizer_kwargs={"learning_rate": 0.01},
        lambda_=1.0,
        max_iter=100,
        random_start=True,
)

# создаем атаку FGSM
attack = FastGradientMethod(estimator=estimator, eps=0.1)

# генерируем атакующий пример
x_test_adv = attack.generate(x_test)


In [ ]:
# Step 8: Inspect results

# orig 7, guide 6
plt.imshow(x_test_adv[3,...].squeeze())

In [ ]:
plt.imshow(testx[3,...].squeeze())

In [ ]:
def plot_gallery(images, titles, h=62, w=47, n_row=3, n_col=4):
    """Helper function to plot a gallery of portraits"""
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i+1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title(np.argmax(titles[i]), size=12)
        plt.xticks(())
        plt.yticks(())

In [ ]:
model.predict(x_test_adv).shape

In [ ]:
plot_gallery(testx, testy)

In [ ]:
plot_gallery(x_test_adv, model.predict(x_test_adv))

NameError: name 'plot_gallery' is not defined